# Cheatsheets
https://wch.github.io/latexsheet/ - Latex

# Notebook Link
TBD

In [76]:
 from IPython.display import HTML, display
def display_table(data):
     display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )))

# Task 1 - Prisoner's Dilemma

The Prisoner’s Dilemma was one of the earliest “games” developed in game theory.

By simulating the Prisoner’s Dilemma we are given an excellent method of studying the issues of conflict vs.
cooperation between individuals. Since the Prisoner’s Dilemma is so basic, it can be used as a model for
various schools of thought, from economics to military strategy to zoology, and even Artificial Intelligence.

1. An agent only cares about itself (it is naturally selfish)

2. Cooperation can be mutually beneficial for all involved

A static 2×2 game represents various combinations of individual choices to cooperate
or defect.

Scenario:

• You and an accomplice are arrested on suspicion of committing some nasty crime.

• The police have been unable to produce enough evidence to convict you of that offense.

• You and an accomplice are held in separate cells. You are not allowed to communicate with each other at
all.

• Each prisoner is told the following: We have arrested you and another person for committing this crime
together.

If you both confess, we will reward your assistance to us, by sentencing you both lightly: 2 years in prison.
If you confess, and the other person does not, we will show our appreciation to you by letting you go. We
will then use your testimony to put the other person in prison for 10 years.

If you both don’t confess, we will not be able to convict you, but we will be able to hold you here and make
you as uncomfortable as we can for 30 days.
If you don't confess, and the other person does, that person's testimony will be used to put you in prison
for 10 years; your accomplice will go free in exchange for the testimony.
Each of you is being given the same deal. Think about it.

A static 2×2 game represents various combinations of individual choices to cooperate or defect.
A canonical payoff matrix:

• R is the “reward” both players receive if they both cooperate

• P is the “punishment” payoff both players receive if they both defect

• P1 receives the “temptation” payoff T if P1 defects while P2 cooperates (P2 receives the “sucker’s” payoff
S)

• Similarly, if P1 cooperates while P2 defects, then P1 receives S while P2 receives T

### Constructing the system

In [84]:
import numpy as np
import copy

def get_cell(m,i,j,x,y):
    if i >= x : i = 0
    if i < 0: i = x-1
    if j >= y: j = 0
    if j < 0 : j = y - 1
    return m[i,j]
    
def utility(m,r,s,t,p,x,y,neighbors):
    u = np.zeros((x, y))
    for i in range(x):
        for j in range(y):
            for ng in neighbors:
                cell = get_cell(m,i+ng[0],j+ng[1],x,y)
                u[i,j] += m[i,j]*cell*p + (1-cell*t) + (1-m[i,j]) * (cell*s + (1-cell)*r)
    return u

def pick_strategy(m,i,j,x,y,neighbors,u):
    max_cell = u[i,j]
    max_coords = [i,j]
    for ng in neighbors:
        cell = get_cell(u,i+ng[0],j+ng[1],x,y)
        if cell > max_cell: 
            max_cell = cell
            max_coords = [i+ng[0],j+ng[1]]
    best_strategy = get_cell(m,max_coords[0],max_coords[1],x,y)
    return best_strategy

def update_strategies(m,x,y,neighbors,u):
    for i in range(x):
        for j in range(y):
            m[i,j] = pick_strategy(m,i,j,x,y,neighbors,u)
    return m

def iterate_pd(x,y,r,s,t,p,max_iterations,debug=False):
    ms = []
    neighbors = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
    m = np.random.randint(2, size=(x, y))
    ms.append(copy.copy(m))
    if debug:
        print(f"m [{0}]:\n{m}")
    for i in range(max_iterations):
        u = utility(m,r,s,t,p,x,y,neighbors)
        m = update_strategies(m,x,y,neighbors,u)
        ms.append(copy.copy(m))
        if debug:
            print(f"u [{i}]:\n{u}")
            print(f"m [{i+1}]:\n{m}")         
    return ms

### Sub task 1 - Investigate the behavior of the system when R = 3; S = 2; T = 4; P = 0

In [93]:
m = iterate_pd(x=5,y=5,r=3,s=2,t=4,p=0,max_iterations=2,debug=False)

for ite,val in enumerate(m):
    print(f"iteration:{ite}")
    display_table(val)

iteration:0


0,1,0,0,1
1,1,1,0,0
1,0,1,1,0
1,1,1,1,0
0,0,1,0,1


iteration:1


0,0,0,0,0
0,0,0,0,0
0,1,0,0,0
0,0,0,0,0
0,0,0,0,0


iteration:2


0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0


### Sub task 2 - Try to use different values of R, S, T, P – observe the formation of different patterns. Use Wikipedia for searching different combinations of these parameters.

We try $r = 3$, $s = 2$, $t = 4$ and $p = 10$ to observe a game where the punishment is really high, suggesting to cooperate

In [94]:
m = iterate_pd(x=5,y=5,r=3,s=2,t=4,p=10,max_iterations=2,debug=False)

for ite,val in enumerate(m):
    print(f"iteration:{ite}")
    display_table(val)

iteration:0


1,0,1,1,1
0,0,1,0,1
0,0,1,0,1
0,0,0,1,1
0,1,0,0,0


iteration:1


1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
1,0,1,1,1
1,1,1,1,1


iteration:2


1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
1,1,1,1,1
1,1,1,1,1


# Task 2 - Beddington DeAngelis Model

Add description here

### Constructing the system

In [101]:
import numpy as np
import copy
    
def get_cell_zero_flux(m,i,j,x,y):
    if i >= x : return 0
    if i < 0: return 0
    if j >= y: return 0
    if j < 0 : return 0
    return m[i,j]

def get_laplacian(n,i,j,x,y,neighbors,h):
    total = 0
    for ng in neighbors:
        cell = get_cell_zero_flux(n,i+ng[0],j+ng[1],x,y)
        total += cell
    return (total - 4*n[i,j]) / h**2


def f_n(n,p,d11,d12,t,r,e,beta,k,ni,w,b):
    delta2n = 0 # ???
    delta2p = 0 # ???
    return r*(1-n/k)*n - beta*n/(b+n+w*p)*p + d11*delta2n + d12*delta2p

def f_p(n,p,d21,d22,t,r,e,beta,k,ni,w,b):
    delta2n = 0 # ???
    delta2p = 0 # ???
    return e*beta*n/(b+n+w*p)*p - ni*p + d21*delta2n + d22*delta2p 

def next_n(n,p,laplacian,d11,d12,t,r,e,beta,k,ni,w,b):
    return n + t*d11*laplacian*n + t*d12*laplacian*n + t*f_n(n,p,d11,d12,t,r,e,beta,k,ni,w,b)

def next_p(n,p,laplacian,d21,d22,t,r,e,beta,k,ni,w,b):
    return p + t*d21*laplacian*n + t*d22*laplacian*p + t*f_p(n,p,d21,d22,t,r,e,beta,k,ni,w,b)

def generate_system(x,y,h,d11,d12,d21,d22,t,r,e,beta,k,ni,w,b,ite):
    p = np.random.randint(2, size=(x, y))
    n = np.random.randint(2, size=(x, y))
    ps = []
    ns = []
    ps.append(copy.copy(p))
    ns.append(copy.copy(n))
    neighbors = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]

    for it in range(ite):
        for i in range(x):
            for j in range(y):
                laplacian = get_laplacian(n,i,j,x,y,neighbors,h)
                n0 = get_cell(n,i,j,x,y)
                p0 = get_cell(p,i,j,x,y)
                p[i,j] = next_p(n0,p0,laplacian,d21,d22,t,r,e,beta,k,ni,w,b)
                n[i,j] = next_n(n0,p0,laplacian,d11,d12,t,r,e,beta,k,ni,w,b)
        ps.append(copy.copy(p))
        ns.append(copy.copy(n))
    return ps,ns
            
ps,ns = generate_system(x=5,y=5,h=0.25,d11=0.01,d12=0.0115,d21=0.01,d22=1,
                        t=0.01,r=0.5,e=1,beta=0.6,k=2.6,ni=0.25,w=0.4,b=0.3154,ite=2)

for ite,val in enumerate(ns):
    print(f"iteration:{ite}")
    display_table(val)

iteration:0


0,1,1,0,0
0,1,1,1,0
1,1,0,0,0
1,1,0,1,1
0,0,1,0,0


iteration:1


0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0


iteration:2


0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0
0,0,0,0,0


### Sub task 1 - Construct the numerical scheme for the Beddington DeAngelis model. Simulate and replicate the results shown in Figure 1.

### Sub task 2 - Experiment with different distributions of random numbers in the initial concentration of predators and preys. Observe the differences in the patterns.

### Sub task 2 - Try to change the parameters of the system and observe how these changes result into different types of patterns.